In [1]:
using GeometricIntegrators

┌ Info: Recompiling stale cache file /Users/mkraus/.julia/compiled/v1.2/GeometricIntegrators/fhpp1.ji for GeometricIntegrators [dcce2d33-59f6-5b8d-9047-0defad88ae06]
└ @ Base loading.jl:1240


In [2]:
function getTableauVSPARKSymplecticProjection(name, q::CoefficientsRK{T}, p::CoefficientsRK{T}, d=[]) where {T}

    la = getCoefficientsLobIIIA2()
    lb = getCoefficientsLobIIIB2()

    @assert q.s == p.s
    @assert q.b == p.b
    @assert q.c == p.c
    
    @assert la.s == lb.s
    @assert la.b == lb.b
#    @assert la.c == lb.c

    s = q.s
    σ = la.s
    ρ = 0

    a_q = q.a
    a_p = p.a
    b_q = q.b
    b_p = p.b
    
    β_q = la.b
    β_p = lb.b
#    β_q = [0.5, R∞*0.5]
#    β_p = [0.5, R∞*0.5]

    α_q = zeros(T, s, 2)
    α_q[:,1] .= 0.5

    α_p = zeros(T, s, 2)
    α_p[:,1] .= 0.5

#    α_q = zeros(T, s, σ)
#    α_p = zeros(T, s, σ)
#    for i in 1:s
#        for j in 1:σ
#            α_q[i,j] = #q.b[i] / β[1] * ( β[1] - α[i,1] )
#            α_p[i,j] = #p.b[i] / β[1] * ( β[1] - α[i,1] )
#        end
#    end

    a_q̃ = zeros(T, σ, s)
    a_p̃ = zeros(T, σ, s)
    for i in 1:σ
        for j in 1:s
            a_q̃[i,j] = b_q[j] / β_p[i] * (β_p[i] - α_p[j,i])
            a_p̃[i,j] = b_p[j] / β_q[i] * (β_q[i] - α_q[j,i])
        end
    end

    α_q̃ = la.a
    α_p̃ = lb.a

    c_q = q.c
    c_p = p.c
    c_λ = la.c
    d_λ = zeros(T, ρ, σ)

    ω_λ = [0.5 0.5 0.0
            0.0 0.0 1.0]


    if length(d) == 0
        return TableauVSPARKprimary(name, min(q.o, p.o),
                            a_q, a_p, α_q, α_p,
                            a_q̃, a_p̃, α_q̃, α_p̃,
                            b_q, b_p, β_q, β_p,
                            c_q, c_p, c_λ, d_λ,
                            ω_λ)
    else
        @assert length(d) == s

        return TableauVSPARKprimary(name, o,
                            a_q, a_p, α_q, α_p,
                            a_q̃, a_p̃, α_q̃, α_p̃,
                            b_q, b_p, β_q, β_p,
                            c_q, c_p, c_λ, d_λ,
                            ω_λ, d)
    end

end

getTableauVSPARKSymplecticProjection (generic function with 2 methods)

In [3]:
function getTableauVSPARKGLRKpSymplectic(s)
    glrk = getCoefficientsGLRK(s)
    getTableauVSPARKSymplecticProjection(Symbol("VSPARK", s, "pSymplectic"), glrk, glrk)
end

getTableauVSPARKGLRKpSymplectic (generic function with 1 method)

In [21]:
function compute_symplecticity_error(a, b, α, β)
    @assert length(b) == size(α, 1) == size(a,2)
    @assert length(β) == size(α, 2) == size(a,1)
    [b[i] * α[i,j] + β[j] * a[j,i] - b[i] * β[j] for i in 1:length(b), j in 1:length(β)]
end

compute_symplecticity_error (generic function with 1 method)

In [5]:
tab = getTableauVSPARKGLRKpSymplectic(1)

AbstractTableauVSPARK{:vspark_primary,Float64}(:VSPARK1pSymplectic, 2, 1, 2, 2, Additive Runge-Kutta Coefficients VSPARK1pSymplecticwith 1 internal stages, 2 projective stages and order 2  a = [0.5]  b = [1.0]  c = [0.5]  α = [0.5 0.0]  β = [0.5, 0.5], Additive Runge-Kutta Coefficients VSPARK1pSymplecticwith 1 internal stages, 2 projective stages and order 2  a = [0.5]  b = [1.0]  c = [0.5]  α = [0.5 0.0]  β = [0.5, 0.5], Projective Runge-Kutta Coefficients VSPARK1pSymplecticwith 1 internal stages, 2 projective stages and order 2  a = [0.0; 1.0]  c = [0.0, 1.0]  α = [0.0 0.0; 0.5 0.5], Projective Runge-Kutta Coefficients VSPARK1pSymplecticwith 1 internal stages, 2 projective stages and order 2  a = [0.0; 1.0]  c = [0.0, 1.0]  α = [0.5 0.0; 0.5 0.0], Multiplier Runge-Kutta coefficients VSPARK1pSymplecticwith 2 projective stages  b = [0.0, 0.0]  c = [0.0, 1.0], [0.5 0.5 0.0; 0.0 0.0 1.0], #undef)

In [22]:
compute_symplecticity_error(tab.q.a, tab.q.b, tab.p.a, tab.p.b)

1×1 Array{Float64,2}:
 0.0

In [23]:
compute_symplecticity_error(tab.p̃.a, tab.p.b, tab.q.α, tab.q.β)

1×2 Array{Float64,2}:
 0.0  0.0

In [26]:
compute_symplecticity_error(tab.q̃.a, tab.q.b, tab.p.α, tab.p.β)

1×2 Array{Float64,2}:
 0.0  0.0

In [27]:
compute_symplecticity_error(tab.q̃.α, tab.q.β, tab.p̃.α, tab.p.β)

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0